In [83]:
from sklearn.datasets import load_breast_cancer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import pandas as pd

In [84]:
data = load_breast_cancer()
X = data['data']
y = data['target']

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [118]:
lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(store_covariance=True)

In [89]:
lda.coef_

array([[ 3.80341976e+00, -5.39212301e-02, -4.39436335e-01,
        -6.34042278e-03,  7.93926949e+00,  9.65027592e+01,
        -1.94072071e+01, -9.48433359e+01,  6.52894977e+00,
        -1.12179949e+02, -8.34003815e+00,  2.43731481e-01,
         1.59124190e-01,  2.40224154e-02, -3.48805766e+02,
         4.26007772e+01,  8.24576776e+01, -3.50357849e+02,
         2.30465093e+01,  5.81140729e+01, -4.13737425e+00,
        -1.85553514e-01,  1.68084303e-01,  1.85463710e-02,
        -2.55422830e+00, -1.47149164e+01, -1.18801082e+01,
         2.55875979e+01, -1.97016750e+01, -2.45735089e+01]])

In [90]:
pred = lda.predict(X_test)

In [94]:
lda.means_

array([[1.74169231e+01, 2.14923077e+01, 1.15012959e+02, 9.75013609e+02,
        1.02531124e-01, 1.43885148e-01, 1.59455266e-01, 8.67635503e-02,
        1.93533136e-01, 6.26227219e-02, 6.00757988e-01, 1.20041598e+00,
        4.28259763e+00, 7.18094675e+01, 6.75819527e-03, 3.17857870e-02,
        4.18483432e-02, 1.50038935e-02, 2.06236627e-02, 3.97157988e-03,
        2.10274556e+01, 2.92200592e+01, 1.40713964e+02, 1.41022781e+03,
        1.44440769e-01, 3.71363373e-01, 4.51448994e-01, 1.81149467e-01,
        3.26636095e-01, 9.11269822e-02],
       [1.21680559e+01, 1.78216434e+01, 7.82140909e+01, 4.64910839e+02,
        9.17334615e-02, 7.98258741e-02, 4.72053007e-02, 2.55395140e-02,
        1.73751049e-01, 6.28359790e-02, 2.84577273e-01, 1.20402867e+00,
        2.01659545e+00, 2.13169266e+01, 7.12550350e-03, 2.20011573e-02,
        2.74909122e-02, 1.00562413e-02, 2.05438776e-02, 3.73115490e-03,
        1.34032587e+01, 2.33585664e+01, 8.72421678e+01, 5.61890210e+02,
        1.23904301e-01,

In [97]:
class1 = X_train[y_train==1, :]

In [137]:
class0 = X_train[y_train==0, :]

In [156]:
np.alltrue((lda.priors_[0] * np.cov(class0.T, ddof=0) + lda.priors_[1] * np.cov(class1.T, ddof=0)) == lda.covariance_)

True

In [163]:
np.allclose(lda.xbar_ ,X_train.mean(axis=0))

True

In [165]:
lda.means_.shape

(2, 30)

In [162]:
X_train.mean(axis=0)

array([1.41176352e+01, 1.91850330e+01, 9.18822418e+01, 6.54377582e+02,
       9.57440220e-02, 1.03619319e-01, 8.88981451e-02, 4.82798703e-02,
       1.81098681e-01, 6.27567692e-02, 4.02015824e-01, 1.20268681e+00,
       2.85825341e+00, 4.00712989e+01, 6.98907473e-03, 2.56354484e-02,
       3.28236723e-02, 1.18939407e-02, 2.05735121e-02, 3.82045560e-03,
       1.62351033e+01, 2.55356923e+01, 1.07103121e+02, 8.76987033e+02,
       1.31532132e-01, 2.52741802e-01, 2.74594569e-01, 1.14182222e-01,
       2.90502198e-01, 8.38678462e-02])

In [154]:
lda.covariance_[0]

array([ 6.04228948e+00,  1.20539129e-01,  4.05295740e+01,  6.10380876e+02,
       -4.73158249e-03,  9.98255704e-03,  4.39274720e-02,  3.40574177e-02,
       -9.65809074e-03, -8.27441332e-03,  2.78826409e-01, -1.71767204e-01,
        2.01724877e+00,  5.80262272e+01, -1.71486953e-03, -1.51562701e-03,
       -8.30639256e-04,  1.38330789e-03, -2.78787171e-03, -1.03777153e-03,
        7.10736302e+00, -1.02913267e+00,  4.79266935e+01,  8.46717039e+02,
       -1.52372115e-02, -1.99819570e-02,  2.22271372e-02,  3.59149565e-02,
       -3.29949785e-02, -1.56847273e-02])

In [103]:
lda.priors_

array([0.37142857, 0.62857143])

In [105]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([169, 286], dtype=int64))

In [108]:
286/sum([169, 286])

0.6285714285714286

In [117]:
X_train.mean(axis=0)

array([1.41176352e+01, 1.91850330e+01, 9.18822418e+01, 6.54377582e+02,
       9.57440220e-02, 1.03619319e-01, 8.88981451e-02, 4.82798703e-02,
       1.81098681e-01, 6.27567692e-02, 4.02015824e-01, 1.20268681e+00,
       2.85825341e+00, 4.00712989e+01, 6.98907473e-03, 2.56354484e-02,
       3.28236723e-02, 1.18939407e-02, 2.05735121e-02, 3.82045560e-03,
       1.62351033e+01, 2.55356923e+01, 1.07103121e+02, 8.76987033e+02,
       1.31532132e-01, 2.52741802e-01, 2.74594569e-01, 1.14182222e-01,
       2.90502198e-01, 8.38678462e-02])

In [126]:
np.cov(X_train,)/X_train.shape[0]

array([[ 11.88091078,  70.28598927,  11.72933594, ...,  26.68844124,
         30.34848587,  20.56758669],
       [ 70.28598927, 426.83773003,  69.08701018, ..., 158.02538386,
        182.38088368, 121.9995804 ],
       [ 11.72933594,  69.08701018,  11.60669297, ...,  26.44940057,
         29.88200846,  20.33577591],
       ...,
       [ 26.68844124, 158.02538386,  26.44940057, ...,  60.91468933,
         68.29570918,  46.60544606],
       [ 30.34848587, 182.38088368,  29.88200846, ...,  68.29570918,
         78.28943866,  52.66223585],
       [ 20.56758669, 121.9995804 ,  20.33577591, ...,  46.60544606,
         52.66223585,  35.78854373]])

In [134]:
(np.linalg.inv(lda.covariance_)).dot(lda.means_[0].T)

array([ 5.75253064e+01, -4.04550731e-01,  9.42562114e+00, -9.17354375e-01,
       -1.62413462e+02, -3.07018153e+03,  2.95993275e+02, -4.92171899e+02,
        5.08396651e+02,  1.80087552e+04,  3.69151815e+01, -1.57302603e+01,
       -5.86467743e+00,  5.81703701e-01,  1.91807097e+03,  1.66896778e+03,
       -3.18468555e+02,  1.33457246e+03,  7.31894505e+02, -1.26198141e+04,
       -1.46923070e+01,  3.03863830e+00,  1.24280836e+00,  4.94055866e-02,
        7.31078127e+02,  1.86560536e+02, -3.94592933e+00, -2.03354141e+02,
       -5.94187619e+01, -1.24631799e+03])

In [136]:
(np.linalg.inv(lda.covariance_)).dot(lda.means_.mean(axis=0).T)

array([ 5.94354124e+01, -4.31630378e-01,  9.20493291e+00, -9.20538583e-01,
       -1.58426302e+02, -3.02171712e+03,  2.86246830e+02, -5.39802934e+02,
        5.11675538e+02,  1.79524176e+04,  3.27267518e+01, -1.56078565e+01,
       -5.78476407e+00,  5.93767939e-01,  1.74289809e+03,  1.69036221e+03,
       -2.77057691e+02,  1.15862012e+03,  7.43468635e+02, -1.25906288e+04,
       -1.67701274e+01,  2.94545194e+00,  1.32722155e+00,  5.87197133e-02,
        7.29795374e+02,  1.79170595e+02, -9.91220886e+00, -1.90503857e+02,
       -6.93130910e+01, -1.25865899e+03])

In [127]:
lda.coef_

array([[ 3.80341976e+00, -5.39212301e-02, -4.39436335e-01,
        -6.34042278e-03,  7.93926949e+00,  9.65027592e+01,
        -1.94072071e+01, -9.48433359e+01,  6.52894977e+00,
        -1.12179949e+02, -8.34003815e+00,  2.43731481e-01,
         1.59124190e-01,  2.40224154e-02, -3.48805766e+02,
         4.26007772e+01,  8.24576776e+01, -3.50357849e+02,
         2.30465093e+01,  5.81140729e+01, -4.13737425e+00,
        -1.85553514e-01,  1.68084303e-01,  1.85463710e-02,
        -2.55422830e+00, -1.47149164e+01, -1.18801082e+01,
         2.55875979e+01, -1.97016750e+01, -2.45735089e+01]])

In [100]:
class1.mean(axis=0) == lda.means_[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])